In [1]:
!{sys.executable} -m pip install sympy

zsh:1: parse error near `-m'


In [22]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import sympy as sp
from sympy import *
import math
import re



with open("equation.txt") as f:
    for line in f:
        e = line #e is the equation

#pe is the proper equation format in np
pe = e

functions = ['sin', 'cos', 'tan', 'exp', 'log', 'sqrt']

for f in functions:
    pe = re.sub(rf'\b{f}\b', f'np.{f}', pe)
pe = pe.replace('^', '**')

se = pe.replace('np.', '')

print(e)
print(pe)
print(se)

def make_function(ex): #pe
    allowed_names = {
        'np': np,
        'x': None  # placeholder, replaced at call time
    }

    def f(x):
        local_dict = allowed_names.copy()
        local_dict['x'] = x
        return eval(ex, {"__builtins__": {}}, local_dict)

    return f



sin(x)+tan(x^2)
np.sin(x)+np.tan(x**2)
sin(x)+tan(x**2)


In [23]:
x = sp.symbols('x')
f = sp.sympify(se)

TAYLOR_NUM = 10
taylors = f.subs(x,0)
diffed = f
for i in range(1, TAYLOR_NUM):
    diffed = diffed.diff(x)
    taylors += diffed.subs(x,0) * 1/(factorial(i))*(x**i)

print(taylors)

x**9/362880 - x**7/5040 + x**6/3 + x**5/120 - x**3/6 + x**2 + x


In [ ]:
original_function = make_function(pe) #The Original


x = np.linspace(-10, 10, 100)
y = ff(x)

fig, ax = plt.subplots()
plt.subplots_adjust(bottom=0.25)

scale0 = 1.0
[line] = ax.plot(x, scale0 * y, label='scale · (sin(x) + x)')
ax.set_title(r"Graph of $y =$"+e)
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.grid(True)
ax.legend()

ax_slider = plt.axes([0.2, 0.1, 0.6, 0.03])
scale_slider = Slider(
    ax=ax_slider,
    label='Scale',
    valmin=0.1,
    valmax=5.0,
    valinit=scale0
)

def update(val):
    scale = scale_slider.val
    line.set_ydata(scale * y)
    fig.canvas.draw_idle()

scale_slider.on_changed(update)

plt.show()


SympifyError: SympifyError: "cannot sympify object of type <class 'function'>"